In [1]:
import pandas as pd
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
import pickle
import pymongo

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Step 1: Load and preprocess the dataset
data = pd.read_csv('train_data.csv')
text = data['Word'].values
labels = data['Sentiment'].values

# Step 2: Convert text data into numerical feature vectors
vectorizer = CountVectorizer()
features = vectorizer.fit_transform(text)

# Step 3: Train the Naive Bayes model
nb = MultinomialNB()
nb.fit(features, labels)

# Step 4: Predict sentiment on new data
new_text = ["I love this movie!", "This product is terrible.", "The food was delicious."]
new_features = vectorizer.transform(new_text)
new_predictions = nb.predict(new_features)
print(new_predictions)

# Step 5: Generate the classification report to evaluate the model
predictions = nb.predict(features)
print(classification_report(labels, predictions))

['Positive' 'Negative' 'Positive']
              precision    recall  f1-score   support

    Negative       1.00      0.99      0.99       210
    Positive       0.99      1.00      0.99       230

    accuracy                           0.99       440
   macro avg       0.99      0.99      0.99       440
weighted avg       0.99      0.99      0.99       440



In [3]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["Feedback"]
mycol = mydb["feedback_data"]
myquery = {}

feedback_data = mycol.find(myquery)
feedback_df = pd.DataFrame(feedback_data)
del feedback_df['_id']
feedback_df = feedback_df.astype({'feedback': str})
feedback_df.head()

,feedback_id,manager_id,date,employee_id,feedback,sentiment
0,1,9002,2021-11-24,385,Meritorious outside Deformed strong war should...,Negative
1,2,9004,2021-05-30,135,Patient next Scream Fetching. Malicious wait R...,Negative
2,3,9002,2021-10-22,727,Protect Intellectual list store son chair job....,Positive
3,4,9001,2021-12-12,93,Unwelcome Honorable Objectionable Boring sit H...,Negative
4,5,9002,2021-06-13,259,Physical Threatening student Clumsy serious Be...,Positive


In [4]:
new_text = feedback_df.feedback
new_features = vectorizer.transform(new_text)
new_predictions = nb.predict(new_features)


In [5]:
new_predictions_df = pd.DataFrame(new_predictions)
pred_counts = new_predictions_df.value_counts()
pred_df = pd.DataFrame(pred_counts)
pred_df.columns = ['count']
print(pred_df)


          count
0              
Positive    512
Negative    488


In [6]:
feedback_df['sentiment'] = new_predictions
feedback_df.head()

,feedback_id,manager_id,date,employee_id,feedback,sentiment
0,1,9002,2021-11-24,385,Meritorious outside Deformed strong war should...,Negative
1,2,9004,2021-05-30,135,Patient next Scream Fetching. Malicious wait R...,Negative
2,3,9002,2021-10-22,727,Protect Intellectual list store son chair job....,Positive
3,4,9001,2021-12-12,93,Unwelcome Honorable Objectionable Boring sit H...,Negative
4,5,9002,2021-06-13,259,Physical Threatening student Clumsy serious Be...,Positive


In [7]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["Feedback"]
mycol = mydb["feedback_data"]

for x in range(len(feedback_df)):
    query = {'feedback_id': x+1}
    record = feedback_df.iloc[x].to_dict()
    update_record = { '$set': record }
    foo = mycol.update_one(query, update_record)

In [8]:
file_name = 'sentiment_model.pkl'
with open(file_name, 'wb') as file:
    pickle.dump(nb, file)